# 03_data_matching.ipynb - Матчинг готових даних

## 🔗 Зіставлення push-користувачів з конверсіями

Цей ноутбук завантажує готові дані з попередніх ноутбуків та виконує їх матчинг по GADID.

### Вхідні дані:
- **`push_users_data.parquet`** - 3,132,011 користувачів з push-даними (з 01_eda_statistic)
- **`conversion_users_data.parquet`** - 166,103 користувачів з конверсіями (з 02_eda_keitaro)

### Результат:
- Матчинг по GADID
- Підготовка даних для A/B та гео аналізу

---

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🔗 МАТЧИНГ ГОТОВИХ ДАНИХ")
print("=" * 50)
print(f"📂 Завантажуємо збережені файли з попередніх ноутбуків")
print(f"🎯 Мета: Швидкий матчинг без повторних запитів до БД")

🔗 МАТЧИНГ ГОТОВИХ ДАНИХ
📂 Завантажуємо збережені файли з попередніх ноутбуків
🎯 Мета: Швидкий матчинг без повторних запитів до БД


## **КРОК 1: Перевірка наявності файлів**

In [2]:
print("\n🔗 КРОК 1: ПЕРЕВІРКА НАЯВНОСТІ ФАЙЛІВ")
print("-" * 40)

# Перевіряємо чи існують файли
files_to_check = {
    'push_data': '../data/processed/push_users_data.parquet',
    'conversion_data': '../data/processed/conversion_users_data.parquet',
    'push_summary': '../outputs/tables/push_summary_stats.json',
    'conversion_summary': '../outputs/tables/conversion_summary_stats.json'
}

available_files = {}
missing_files = []

for name, path in files_to_check.items():
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / 1024 / 1024
        available_files[name] = path
        print(f"✅ {name}: {path} ({size_mb:.1f} MB)")
    else:
        missing_files.append(name)
        print(f"❌ {name}: {path} - файл не знайдено")

if missing_files:
    print(f"\n⚠️ Відсутні файли: {missing_files}")
    print(f"💡 Запустіть спочатку ноутбуки 01_eda_statistic та 02_eda_keitaro")
    raise FileNotFoundError(f"Відсутні необхідні файли: {missing_files}")
else:
    print(f"\n✅ Всі необхідні файли знайдено!")


🔗 КРОК 1: ПЕРЕВІРКА НАЯВНОСТІ ФАЙЛІВ
----------------------------------------
✅ push_data: ../data/processed/push_users_data.parquet (132.8 MB)
✅ conversion_data: ../data/processed/conversion_users_data.parquet (9.3 MB)
✅ push_summary: ../outputs/tables/push_summary_stats.json (0.0 MB)
✅ conversion_summary: ../outputs/tables/conversion_summary_stats.json (0.0 MB)

✅ Всі необхідні файли знайдено!


## **КРОК 2: Завантаження push-даних**

In [ ]:
print("\n🔗 КРОК 2: ЗАВАНТАЖЕННЯ PUSH-ДАНИХ З КОНТРОЛЬНОЮ ГРУПОЮ")
print("-" * 40)

# Використовуємо DataLoader для отримання правильних даних з контрольною групою
import sys
import os
sys.path.append(os.path.abspath('..'))

from src.data_loader import DataLoader

print("📂 Завантаження повного набору push-даних через DataLoader (включаючи контрольну групу)...")
data_loader = DataLoader(cache_enabled=True)

# Завантажуємо повний набір даних включаючи контрольну групу з виправленою логікою
push_df = data_loader.load_complete_dataset(include_control_group=True)

if push_df.empty:
    print("❌ Не вдалося завантажити push-дані!")
    raise ValueError("Push-дані не завантажено")

print(f"✅ Push-дані завантажено: {len(push_df):,} записів")
print(f"📊 Колонки: {list(push_df.columns)}")

# Перевіряємо структуру даних
print("\n📈 СТРУКТУРА PUSH-ДАНИХ З КОНТРОЛЬНОЮ ГРУПОЮ:")
print(f"   📱 Унікальних користувачів (gadid): {push_df['gadid'].nunique():,}")
print(f"   📱 Загальна к-ть push: {push_df['push_count'].sum():,}")
print(f"   📱 Середня к-ть push: {push_df['push_count'].mean():.2f}")
print(f"   📱 Медіанна к-ть push: {push_df['push_count'].median():.0f}")

if 'ab_group' in push_df.columns:
    ab_distribution = push_df['ab_group'].value_counts().sort_index()
    print(f"   🏷️ A/B групи: {ab_distribution.to_dict()}")
    
    # Перевіряємо контрольну групу
    if '6' in ab_distribution.index:
        control_users = ab_distribution['6']
        print(f"   ✅ КОНТРОЛЬНА ГРУПА: {control_users:,} користувачів")
        if control_users > 500000:
            print(f"   ✅ Контрольна група має реалістичний розмір")
        else:
            print(f"   ⚠️ Контрольна група може бути занадто малою")
    else:
        print(f"   ❌ КОНТРОЛЬНА ГРУПА НЕ ЗНАЙДЕНА!")

if 'tier' in push_df.columns:
    print(f"   🌍 Розподіл по tier: {push_df['tier'].value_counts().to_dict()}")
elif 'country' in push_df.columns:
    print(f"   🌍 Топ-5 країн: {push_df['country'].value_counts().head().to_dict()}")

# Показуємо приклад
print("\n🔍 ПРИКЛАД PUSH-ДАНИХ:")
display_cols = ['gadid', 'ab_group', 'push_count', 'country']
if 'avg_success_rate' in push_df.columns:
    display_cols.append('avg_success_rate')
display(push_df[display_cols].head())

# Показуємо статистику по групах
print("\n📊 СТАТИСТИКА ПО A/B ГРУПАХ:")
if 'ab_group' in push_df.columns:
    group_stats = push_df.groupby('ab_group').agg({
        'gadid': 'count',
        'push_count': ['sum', 'mean'],
        'avg_success_rate': 'mean' if 'avg_success_rate' in push_df.columns else 'count'
    }).round(2)
    
    # Додаємо тип групи
    group_stats['group_type'] = group_stats.index.map(lambda x: 'Control Group' if x == '6' else 'Push Group')
    print(group_stats)

print(f"\n✅ Push-дані з контрольною групою успішно завантажено!")

## **КРОК 3: Завантаження конверсійних даних**

In [4]:
print("\n🔗 КРОК 3: ЗАВАНТАЖЕННЯ КОНВЕРСІЙНИХ ДАНИХ")
print("-" * 40)

# Завантажуємо готові конверсійні дані
print("📂 Завантаження conversion_users_data.parquet...")
conversion_df = pd.read_parquet('../data/processed/conversion_users_data.parquet')

print(f"✅ Конверсійні дані завантажено: {len(conversion_df):,} записів")
print(f"📊 Колонки: {list(conversion_df.columns)}")

# Перевіряємо структуру
print("\n📈 СТРУКТУРА КОНВЕРСІЙНИХ ДАНИХ:")
print(f"   👥 Унікальних користувачів (gadid): {conversion_df['gadid'].nunique():,}")

if 'total_deposits' in conversion_df.columns:
    deposit_users = (conversion_df['total_deposits'] > 0).sum()
    print(f"   💰 Користувачі з депозитами: {deposit_users:,}")
    print(f"   💰 Загальна сума депозитів: {conversion_df['total_deposits'].sum():,}")

if 'total_registrations' in conversion_df.columns:
    reg_users = (conversion_df['total_registrations'] > 0).sum()
    print(f"   📝 Користувачі з реєстраціями: {reg_users:,}")
    print(f"   📝 Загальна к-ть реєстрацій: {conversion_df['total_registrations'].sum():,}")

if 'total_revenue' in conversion_df.columns:
    total_revenue = conversion_df['total_revenue'].sum()
    print(f"   💵 Загальний дохід: ${total_revenue:,.2f}")
    print(f"   💵 ARPU: ${total_revenue/len(conversion_df):.2f}")

if 'tier' in conversion_df.columns:
    print(f"   🌍 Розподіл по tier: {conversion_df['tier'].value_counts().to_dict()}")

# TMNT групи якщо є
if 'group_name' in conversion_df.columns:
    print(f"   🐢 TMNT групи: {conversion_df['group_name'].value_counts().to_dict()}")

# Показуємо приклад
print("\n🔍 ПРИКЛАД КОНВЕРСІЙНИХ ДАНИХ:")
display(conversion_df.head())

# Завантажуємо summary
if os.path.exists('../outputs/tables/conversion_summary_stats.json'):
    with open('../outputs/tables/conversion_summary_stats.json', 'r', encoding='utf-8') as f:
        conv_summary = json.load(f)
    print(f"\n📊 SUMMARY З 02_EDA_KEITARO:")
    if 'analysis_info' in conv_summary:
        print(f"   📅 Період: {conv_summary['analysis_info'].get('period', 'N/A')}")
        print(f"   🎯 Цільові застосунки: {conv_summary['analysis_info'].get('target_apps', [])}")
    if 'revenue_metrics' in conv_summary:
        print(f"   💵 Загальний дохід: ${conv_summary['revenue_metrics'].get('total_revenue', 0):,.2f}")
        print(f"   💵 Середній ARPU: ${conv_summary['revenue_metrics'].get('avg_revenue_per_user', 0):.2f}")
    if 'conversion_metrics' in conv_summary:
        print(f"   🎖️ Коефіцієнт депозитів: {conv_summary['conversion_metrics'].get('deposit_rate', 0):.1f}%")


🔗 КРОК 3: ЗАВАНТАЖЕННЯ КОНВЕРСІЙНИХ ДАНИХ
----------------------------------------
📂 Завантаження conversion_users_data.parquet...
✅ Конверсійні дані завантажено: 166,103 записів
📊 Колонки: ['gadid', 'country', 'campaign_id', 'group_id', 'group_name', 'total_deposits', 'total_registrations', 'total_revenue', 'lead_revenue', 'total_events', 'first_conversion_date', 'last_conversion_date', 'first_conversion_datetime', 'last_conversion_datetime', 'conversion_days', 'conversion_type', 'tier', 'conversion_window_days', 'avg_revenue_per_conversion', 'arpu']

📈 СТРУКТУРА КОНВЕРСІЙНИХ ДАНИХ:
   👥 Унікальних користувачів (gadid): 163,788
   💰 Користувачі з депозитами: 24,503
   💰 Загальна сума депозитів: 24,549
   📝 Користувачі з реєстраціями: 141,878
   📝 Загальна к-ть реєстрацій: 142,649
   💵 Загальний дохід: $245,628.07
   💵 ARPU: $1.48
   🌍 Розподіл по tier: {'Tier 2': 104517, 'Tier 3': 29026, 'Other': 20909, 'Tier 1': 11429, 'Unknown': 222}
   🐢 TMNT групи: {'Splinter (Oleksandr)': 67184,

,gadid,country,campaign_id,group_id,group_name,total_deposits,total_registrations,total_revenue,lead_revenue,total_events,first_conversion_date,last_conversion_date,first_conversion_datetime,last_conversion_datetime,conversion_days,conversion_type,tier,conversion_window_days,avg_revenue_per_conversion,arpu
0,94fd2965-34b3-409e-b5ab-3a8372509b44,KZ,2879,1038,Zhenya (Leonardo),1,0,12512.00,0.0,1,2025-05-31,2025-05-31,2025-05-31 19:45:10,2025-05-31 19:45:10,1,Deposit,Other,1,12512.00,12512.00
1,e114b39d-f2c9-469e-ba69-7465f57624a8,LT,2884,1039,Bohdan (Raphael),3,0,460.00,0.0,3,2025-05-25,2025-05-25,2025-05-25 19:15:47,2025-05-25 19:15:47,1,Deposit,Tier 2,1,153.33,460.00
2,57d09d48-fba3-42e6-ac36-3765f83159de,LT,2860,1065,Splinter (Oleksandr),3,0,460.00,0.0,3,2025-05-25,2025-05-25,2025-05-25 10:13:48,2025-05-25 10:13:48,1,Deposit,Tier 2,1,153.33,460.00
3,e671ca48-9655-44d1-99fb-e7fa666a2244,ES,2860,1065,Splinter (Oleksandr),2,0,363.16,0.0,2,2025-05-25,2025-05-30,2025-05-25 16:57:53,2025-05-30 21:02:21,2,Deposit,Tier 2,6,181.58,363.16
4,afd104fb-57dd-4ec6-a1cc-6dff44d2636a,ES,2826,1038,Zhenya (Leonardo),2,0,307.16,0.0,2,2025-05-26,2025-05-27,2025-05-26 21:55:49,2025-05-27 00:02:18,2,Deposit,Tier 2,2,153.58,307.16



📊 SUMMARY З 02_EDA_KEITARO:
   📅 Період: 2025-05-22 - 2025-06-07
   🎯 Цільові застосунки: ['Michelangelo', 'Leonardo', 'Raphael', 'Splinter']
   💵 Загальний дохід: $245,628.07
   💵 Середній ARPU: $1.48
   🎖️ Коефіцієнт депозитів: 14.8%


## **КРОК 4: Матчинг даних по GADID**

In [ ]:
print("\n🔗 КРОК 4: МАТЧИНГ ПО GADID З КОНТРОЛЬНОЮ ГРУПОЮ")
print("-" * 40)

# Аналіз пересічення множин
push_gadids = set(push_df['gadid'])
conv_gadids = set(conversion_df['gadid'])

intersection = push_gadids & conv_gadids
push_only = push_gadids - conv_gadids
conv_only = conv_gadids - push_gadids

print(f"📊 АНАЛІЗ ПЕРЕСІЧЕННЯ GADID:")
print(f"   📱 Користувачі лише з push (включаючи контроль): {len(push_only):,}")
print(f"   💰 Користувачі лише з конверсіями: {len(conv_only):,}")
print(f"   🔗 Користувачі з обома типами даних: {len(intersection):,}")
print(f"   📈 % покриття (від push): {len(intersection)/len(push_gadids)*100:.2f}%")
print(f"   📈 % покриття (від конверсій): {len(intersection)/len(conv_gadids)*100:.2f}%")

# Аналіз контрольної групи окремо
if 'ab_group' in push_df.columns:
    control_df = push_df[push_df['ab_group'] == '6']
    push_treatment_df = push_df[push_df['ab_group'] != '6']
    
    control_gadids = set(control_df['gadid'])
    treatment_gadids = set(push_treatment_df['gadid'])
    
    control_conv_intersection = control_gadids & conv_gadids
    treatment_conv_intersection = treatment_gadids & conv_gadids
    
    print(f"\n🧪 АНАЛІЗ КОНТРОЛЬНОЇ ГРУПИ:")
    print(f"   👥 Користувачів у контрольній групі: {len(control_gadids):,}")
    print(f"   👥 Користувачів у push-групах: {len(treatment_gadids):,}")
    print(f"   🔗 Контроль з конверсіями: {len(control_conv_intersection):,} ({len(control_conv_intersection)/len(control_gadids)*100:.2f}%)")
    print(f"   🔗 Push-групи з конверсіями: {len(treatment_conv_intersection):,} ({len(treatment_conv_intersection)/len(treatment_gadids)*100:.2f}%)")

# Виконуємо LEFT JOIN - всі push користувачі + їх конверсії
print(f"\n🔗 Виконання LEFT JOIN матчингу...")
matched_df = push_df.merge(
    conversion_df, 
    on='gadid', 
    how='left',
    suffixes=('_push', '_conv')
)

print(f"✅ Матчинг завершено: {len(matched_df):,} записів")

# Обробляємо пропуски
numeric_cols = ['total_deposits', 'total_registrations', 'total_revenue']
for col in numeric_cols:
    if col in matched_df.columns:
        matched_df[col] = matched_df[col].fillna(0)

# Створюємо бінарні індикатори
if 'total_deposits' in matched_df.columns:
    matched_df['has_deposit'] = (matched_df['total_deposits'] > 0).astype(int)
if 'total_registrations' in matched_df.columns:
    matched_df['has_registration'] = (matched_df['total_registrations'] > 0).astype(int)

matched_df['has_conversion'] = 0
if 'total_deposits' in matched_df.columns and 'total_registrations' in matched_df.columns:
    matched_df['has_conversion'] = ((matched_df['total_deposits'] > 0) | (matched_df['total_registrations'] > 0)).astype(int)

# Використовуємо найкращі дані по країнах та tier
if 'country_push' in matched_df.columns and 'country_conv' in matched_df.columns:
    matched_df['country_final'] = matched_df['country_push'].fillna(matched_df['country_conv'])
elif 'country' in matched_df.columns:
    matched_df['country_final'] = matched_df['country']

if 'tier_push' in matched_df.columns and 'tier_conv' in matched_df.columns:
    matched_df['tier_final'] = matched_df['tier_push'].fillna(matched_df['tier_conv'])
elif 'tier' in matched_df.columns:
    matched_df['tier_final'] = matched_df['tier']

print(f"\n🎯 РЕЗУЛЬТАТ МАТЧИНГУ З КОНТРОЛЬНОЮ ГРУПОЮ:")
print(f"   👥 Загалом користувачів: {len(matched_df):,}")
if 'has_conversion' in matched_df.columns:
    conv_count = matched_df['has_conversion'].sum()
    conv_rate = matched_df['has_conversion'].mean() * 100
    print(f"   ✅ З конверсіями: {conv_count:,} ({conv_rate:.2f}%)")
if 'has_deposit' in matched_df.columns:
    dep_count = matched_df['has_deposit'].sum()
    dep_rate = matched_df['has_deposit'].mean() * 100
    print(f"   💰 З депозитами: {dep_count:,} ({dep_rate:.2f}%)")
if 'has_registration' in matched_df.columns:
    reg_count = matched_df['has_registration'].sum()
    reg_rate = matched_df['has_registration'].mean() * 100
    print(f"   📝 З реєстраціями: {reg_count:,} ({reg_rate:.2f}%)")

# Окремі статистики для контрольної групи та push-груп
if 'ab_group' in matched_df.columns:
    print(f"\n📊 РОЗПОДІЛ КОНВЕРСІЙ ПО ТИПУ ГРУПИ:")
    
    control_matched = matched_df[matched_df['ab_group'] == '6']
    push_matched = matched_df[matched_df['ab_group'] != '6']
    
    if len(control_matched) > 0 and 'has_deposit' in matched_df.columns:
        control_conv = control_matched['has_deposit'].mean() * 100
        push_conv = push_matched['has_deposit'].mean() * 100 if len(push_matched) > 0 else 0
        
        print(f"   🔴 Контрольна grupa (без push): {len(control_matched):,} користувачів, {control_conv:.2f}% депозитів")
        print(f"   🔵 Push-групи: {len(push_matched):,} користувачів, {push_conv:.2f}% депозитів")
        print(f"   📈 Різниця: {push_conv - control_conv:+.2f} п.п.")

# Показуємо приклад результату
print("\n🔍 ПРИКЛАД МАТЧИНГУ:")
display_cols = ['gadid', 'ab_group', 'push_count']
if 'has_deposit' in matched_df.columns:
    display_cols.append('has_deposit')
if 'total_revenue' in matched_df.columns:
    display_cols.append('total_revenue')
if 'tier_final' in matched_df.columns:
    display_cols.append('tier_final')

# Показуємо приклади з різних груп
print("Приклад з контрольної групи (6):")
control_sample = matched_df[matched_df['ab_group'] == '6'].head(3)
if not control_sample.empty:
    display(control_sample[display_cols])
else:
    print("Контрольна група не знайдена у матчингу")

print("Приклад з push-груп (1-5):")
push_sample = matched_df[matched_df['ab_group'] != '6'].head(5)
if not push_sample.empty:
    display(push_sample[display_cols])
else:
    print("Push-групи не знайдені у матчингу")

## **КРОК 5: Додаткові метрики та категоризація**

In [ ]:
print("\n🔗 КРОК 5: ДОДАТКОВІ МЕТРИКИ З КОНТРОЛЬНОЮ ГРУПОЮ")
print("-" * 40)

# Додаємо корисні метрики (тільки для push-груп, не для контролю)
if 'push_count' in matched_df.columns and 'total_revenue' in matched_df.columns:
    matched_df['revenue_per_push'] = 0
    # Розраховуємо тільки для push-груп (не для контролю)
    push_mask = matched_df['ab_group'] != '6'
    matched_df.loc[push_mask, 'revenue_per_push'] = (
        matched_df.loc[push_mask, 'total_revenue'] / 
        matched_df.loc[push_mask, 'push_count'].clip(lower=1)
    )
    print("✅ Додано revenue_per_push (тільки для push-груп)")

# Категорії користувачів з урахуванням контрольної групи
def categorize_user_with_control(row):
    if row.get('ab_group') == '6':
        # Контрольна група
        if row.get('has_deposit', 0) > 0:
            return 'Control - Depositor'
        elif row.get('has_registration', 0) > 0:
            return 'Control - Registration Only'
        else:
            return 'Control - No Conversion'
    else:
        # Push-групи
        if row.get('has_deposit', 0) > 0:
            return 'Push - Depositor'
        elif row.get('has_registration', 0) > 0:
            return 'Push - Registration Only'
        else:
            return 'Push - No Conversion'

matched_df['user_category'] = matched_df.apply(categorize_user_with_control, axis=1)
print("✅ Додано user_category з розділенням контроль/push")

# Сегменти по кількості push-ів (тільки для push-груп)
if 'push_count' in matched_df.columns:
    matched_df['push_segment'] = 'No Push (Control)'
    push_mask = matched_df['ab_group'] != '6'
    matched_df.loc[push_mask, 'push_segment'] = pd.cut(
        matched_df.loc[push_mask, 'push_count'],
        bins=[0, 1, 5, 10, 20, 50, float('inf')],
        labels=['1', '2-5', '6-10', '11-20', '21-50', '50+']
    ).astype(str)
    print("✅ Додано push_segment (з окремою категорією для контролю)")

# Статистика по категоріях
print("\n👥 РОЗПОДІЛ ПО КАТЕГОРІЯХ (КОНТРОЛЬ vs PUSH):")
category_stats = matched_df['user_category'].value_counts()
category_pct = matched_df['user_category'].value_counts(normalize=True) * 100

for category in category_stats.index:
    count = category_stats[category]
    pct = category_pct[category]
    print(f"   {category}: {count:,} ({pct:.1f}%)")

# A/B аналіз з контрольною групою
if 'ab_group' in matched_df.columns:
    print("\n🏷️ A/B АНАЛІЗ З КОНТРОЛЬНОЮ ГРУПОЮ:")
    ab_stats = matched_df.groupby('ab_group').agg({
        'gadid': 'count',
        'push_count': 'mean'
    }).round(2)
    
    if 'has_deposit' in matched_df.columns:
        ab_stats['deposits'] = matched_df.groupby('ab_group')['has_deposit'].sum()
        ab_stats['deposit_rate'] = (ab_stats['deposits'] / ab_stats['gadid'] * 100).round(3)
    
    if 'total_revenue' in matched_df.columns:
        ab_stats['total_revenue'] = matched_df.groupby('ab_group')['total_revenue'].sum().round(2)
        ab_stats['arpu'] = (ab_stats['total_revenue'] / ab_stats['gadid']).round(4)
    
    # Додаємо тип групи
    ab_stats['group_type'] = ab_stats.index.map(lambda x: 'Control' if x == '6' else 'Push')
    
    print(ab_stats)
    
    # Розраховуємо lift проти контрольної групи
    if '6' in ab_stats.index and 'deposit_rate' in ab_stats.columns:
        control_rate = ab_stats.loc['6', 'deposit_rate']
        print(f"\n📈 LIFT АНАЛІЗ (ПРОТИ КОНТРОЛЬНОЇ ГРУПИ):")
        print(f"   🔴 Контрольна група: {control_rate:.3f}% депозитів")
        
        for group in ab_stats.index:
            if group != '6':
                group_rate = ab_stats.loc[group, 'deposit_rate']
                lift = group_rate - control_rate
                lift_pct = (group_rate / control_rate - 1) * 100 if control_rate > 0 else 0
                print(f"   🔵 Група {group}: {group_rate:.3f}% ({lift:+.3f} п.п., {lift_pct:+.1f}%)")

# Tier аналіз з контрольною групою
if 'tier_final' in matched_df.columns:
    print("\n🌍 TIER АНАЛІЗ З КОНТРОЛЬНОЮ ГРУПОЮ:")
    
    # Загальна статистика
    tier_stats = matched_df.groupby('tier_final').agg({
        'gadid': 'count',
        'push_count': 'mean'
    }).round(2)
    
    if 'has_deposit' in matched_df.columns:
        tier_stats['deposits'] = matched_df.groupby('tier_final')['has_deposit'].sum()
        tier_stats['deposit_rate'] = (tier_stats['deposits'] / tier_stats['gadid'] * 100).round(2)
    
    print("Загальна статистика по tier:")
    print(tier_stats)
    
    # Статистика по tier та типу групи
    if 'ab_group' in matched_df.columns:
        print("\nСтатистика по tier та типу групи:")
        tier_group_stats = matched_df.groupby(['tier_final', 'ab_group']).agg({
            'gadid': 'count',
            'has_deposit': 'sum' if 'has_deposit' in matched_df.columns else 'count'
        })
        
        if 'has_deposit' in matched_df.columns:
            tier_group_stats['deposit_rate'] = (
                tier_group_stats['has_deposit'] / tier_group_stats['gadid'] * 100
            ).round(3)
        
        # Показуємо топ tier-и
        for tier in ['Tier 1', 'Tier 2', 'Tier 3']:
            if tier in tier_group_stats.index:
                print(f"\n{tier}:")
                tier_data = tier_group_stats.loc[tier]
                if len(tier_data.shape) > 1:
                    tier_data['group_type'] = tier_data.index.map(lambda x: 'Control' if x == '6' else 'Push')
                    print(tier_data)
                else:
                    print(tier_data)

## **КРОК 6: Збереження результатів матчингу**

In [ ]:
print("\n🔗 КРОК 6: ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТІВ З КОНТРОЛЬНОЮ ГРУПОЮ")
print("-" * 40)

# Створюємо папки
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../outputs/tables', exist_ok=True)

# Основний файл матчингу з контрольною групою
matched_df.to_parquet('../data/processed/matched_push_conversions_with_control.parquet', index=False)
print("✅ Основний файл: matched_push_conversions_with_control.parquet")

# Файл для A/B аналізу з контрольною групою
if 'ab_group' in matched_df.columns:
    ab_ready_cols = ['gadid', 'ab_group', 'push_count']
    
    # Додаємо доступні колонки
    optional_cols = ['has_deposit', 'has_registration', 'total_revenue', 'tier_final', 'user_category', 'push_segment']
    for col in optional_cols:
        if col in matched_df.columns:
            ab_ready_cols.append(col)
    
    ab_ready = matched_df[ab_ready_cols].copy()
    
    # Додаємо тип групи для зручності
    ab_ready['group_type'] = ab_ready['ab_group'].apply(lambda x: 'Control' if x == '6' else 'Push')
    
    ab_ready.to_parquet('../data/processed/ab_analysis_ready_with_control.parquet', index=False)
    ab_ready.to_csv('../outputs/tables/ab_analysis_ready_with_control.csv', index=False)
    print("✅ Файли для A/B аналізу з контролем збережено")

# Файл для гео аналізу з контрольною групою
if 'tier_final' in matched_df.columns:
    geo_ready_cols = ['gadid', 'tier_final', 'ab_group']
    
    # Додаємо доступні колонки
    optional_cols = ['country_final', 'push_count', 'has_deposit', 'total_revenue', 'user_category']
    for col in optional_cols:
        if col in matched_df.columns:
            geo_ready_cols.append(col)
    
    geo_ready = matched_df[geo_ready_cols].copy()
    
    # Додаємо тип групи
    geo_ready['group_type'] = geo_ready['ab_group'].apply(lambda x: 'Control' if x == '6' else 'Push')
    
    geo_ready.to_parquet('../data/processed/geo_analysis_ready_with_control.parquet', index=False)
    geo_ready.to_csv('../outputs/tables/geo_analysis_ready_with_control.csv', index=False)
    print("✅ Файли для гео аналізу з контролем збережено")

# Окремий файл для контрольної групи
if 'ab_group' in matched_df.columns:
    control_data = matched_df[matched_df['ab_group'] == '6'].copy()
    if not control_data.empty:
        control_data.to_parquet('../data/processed/control_group_data.parquet', index=False)
        control_data.to_csv('../outputs/tables/control_group_data.csv', index=False)
        print(f"✅ Контрольна група збережена: {len(control_data):,} користувачів")

# Резюме матчингу з контрольною групою
control_users = len(matched_df[matched_df['ab_group'] == '6']) if 'ab_group' in matched_df.columns else 0
push_users = len(matched_df[matched_df['ab_group'] != '6']) if 'ab_group' in matched_df.columns else len(matched_df)

# Розраховуємо статистики окремо для контролю та push
control_stats = {}
push_stats = {}

if 'ab_group' in matched_df.columns and control_users > 0:
    control_df = matched_df[matched_df['ab_group'] == '6']
    push_df_matched = matched_df[matched_df['ab_group'] != '6']
    
    if 'has_deposit' in matched_df.columns:
        control_stats['deposit_rate'] = float(control_df['has_deposit'].mean() * 100)
        control_stats['total_deposits'] = int(control_df['has_deposit'].sum())
        
        push_stats['deposit_rate'] = float(push_df_matched['has_deposit'].mean() * 100)
        push_stats['total_deposits'] = int(push_df_matched['has_deposit'].sum())
    
    if 'total_revenue' in matched_df.columns:
        control_stats['total_revenue'] = float(control_df['total_revenue'].sum())
        control_stats['arpu'] = float(control_df['total_revenue'].sum() / len(control_df))
        
        push_stats['total_revenue'] = float(push_df_matched['total_revenue'].sum())
        push_stats['arpu'] = float(push_df_matched['total_revenue'].sum() / len(push_df_matched))

matching_summary = {
    'matching_info': {
        'push_users': len(push_df) if 'push_df' in locals() else 0,
        'conversion_users': len(conversion_df) if 'conversion_df' in locals() else 0,
        'matched_users': len(matched_df),
        'overlapping_users': len(intersection) if 'intersection' in locals() else 0,
        'overlap_rate_from_push': len(intersection) / len(push_gadids) * 100 if 'intersection' in locals() and 'push_gadids' in locals() else 0,
        'overlap_rate_from_conversions': len(intersection) / len(conv_gadids) * 100 if 'intersection' in locals() and 'conv_gadids' in locals() else 0,
        'matching_quality': 'excellent' if len(intersection) > 10000 else ('good' if len(intersection) > 1000 else 'poor') if 'intersection' in locals() else 'unknown'
    },
    'control_group_analysis': {
        'users': control_users,
        'percentage_of_total': control_users / len(matched_df) * 100,
        'statistics': control_stats,
        'note': 'Control group receives no push notifications'
    },
    'push_groups_analysis': {
        'users': push_users,
        'percentage_of_total': push_users / len(matched_df) * 100,
        'statistics': push_stats,
        'note': 'Push treatment groups (1-5)'
    },
    'final_metrics': {
        'total_users': len(matched_df),
        'users_with_deposits': int(matched_df['has_deposit'].sum()) if 'has_deposit' in matched_df.columns else 0,
        'users_with_registrations': int(matched_df['has_registration'].sum()) if 'has_registration' in matched_df.columns else 0,
        'deposit_conversion_rate': float(matched_df['has_deposit'].mean() * 100) if 'has_deposit' in matched_df.columns else 0,
        'total_revenue': float(matched_df['total_revenue'].sum()) if 'total_revenue' in matched_df.columns else 0,
        'arpu': float(matched_df['total_revenue'].sum() / len(matched_df)) if 'total_revenue' in matched_df.columns else 0
    },
    'lift_analysis': {},
    'segmentation': {
        'user_categories': matched_df['user_category'].value_counts().to_dict() if 'user_category' in matched_df.columns else {},
        'ab_groups': matched_df['ab_group'].value_counts().to_dict() if 'ab_group' in matched_df.columns else {},
        'tiers': matched_df['tier_final'].value_counts().to_dict() if 'tier_final' in matched_df.columns else {}
    },
    'processing_info': {
        'completed_at': datetime.now().isoformat(),
        'source_files': {
            'push_data': 'DataLoader.load_complete_dataset(include_control_group=True)',
            'conversion_data': 'conversion_users_data.parquet'
        },
        'includes_control_group': True
    }
}

# Додаємо lift аналіз якщо є контрольна група
if 'ab_group' in matched_df.columns and control_users > 0 and 'has_deposit' in matched_df.columns:
    control_rate = control_stats.get('deposit_rate', 0)
    push_rate = push_stats.get('deposit_rate', 0)
    
    matching_summary['lift_analysis'] = {
        'control_deposit_rate': control_rate,
        'push_average_deposit_rate': push_rate,
        'absolute_lift': push_rate - control_rate,
        'relative_lift_percent': (push_rate / control_rate - 1) * 100 if control_rate > 0 else 0,
        'lift_interpretation': 'positive' if push_rate > control_rate else ('negative' if push_rate < control_rate else 'no_effect')
    }

# Зберігаємо резюме
with open('../outputs/tables/matching_summary_with_control.json', 'w', encoding='utf-8') as f:
    json.dump(matching_summary, f, ensure_ascii=False, indent=2)

print("✅ Резюме матчингу з контрольною групою збережено")

# Фінальна статистика
print(f"\n📈 ФІНАЛЬНА СТАТИСТИКА МАТЧИНГУ З КОНТРОЛЬНОЮ ГРУПОЮ:")
print(f"   👥 Загалом користувачів: {len(matched_df):,}")
if 'intersection' in locals():
    print(f"   🔗 Спільних GADID: {len(intersection):,}")
    print(f"   📈 Якість матчингу: {matching_summary['matching_info']['matching_quality']}")

print(f"   🔴 Контрольна група: {control_users:,} користувачів ({control_users/len(matched_df)*100:.1f}%)")
print(f"   🔵 Push-групи: {push_users:,} користувачів ({push_users/len(matched_df)*100:.1f}%)")

if 'has_deposit' in matched_df.columns:
    print(f"   💰 Загальна конверсія: {matched_df['has_deposit'].mean()*100:.2f}%")
    if control_users > 0:
        control_conv = control_stats.get('deposit_rate', 0)
        push_conv = push_stats.get('deposit_rate', 0)
        lift = push_conv - control_conv
        print(f"   📊 Контроль: {control_conv:.2f}%, Push: {push_conv:.2f}%, Lift: {lift:+.2f} п.п.")

if 'total_revenue' in matched_df.columns:
    print(f"   💵 Загальний дохід: ${matched_df['total_revenue'].sum():,.2f}")
    print(f"   💵 ARPU: ${matched_df['total_revenue'].sum()/len(matched_df):.2f}")

print(f"\n✅ МАТЧИНГ З КОНТРОЛЬНОЮ ГРУПОЮ ЗАВЕРШЕНО УСПІШНО!")
print(f"📂 Файли збережено в data/processed/ та outputs/tables/")
print(f"➡️ Готово для A/B аналізу з контрольною групою")

## **КРОК 7: Підготовка до наступних етапів**

In [ ]:
print("\n🔗 КРОК 7: ПІДГОТОВКА ДО A/B АНАЛІЗУ З КОНТРОЛЬНОЮ ГРУПОЮ")
print("-" * 40)

print("📁 СТВОРЕНІ ФАЙЛИ З КОНТРОЛЬНОЮ ГРУПОЮ:")
print("   📊 matched_push_conversions_with_control.parquet - основний файл матчингу")
print("   🏷️ ab_analysis_ready_with_control.* - готові дані для A/B аналізу з контролем")
print("   🌍 geo_analysis_ready_with_control.* - готові дані для гео аналізу з контролем")
print("   🔴 control_group_data.* - окремі дані контрольної групи")
print("   📋 matching_summary_with_control.json - підсумкова статистика")

print("\n🎯 ГОТОВНІСТЬ ДО НАСТУПНИХ НОУТБУКІВ:")

# Перевіряємо готовність для A/B аналізу з контрольною групою
if 'ab_group' in matched_df.columns and matched_df['ab_group'].nunique() > 1:
    ab_groups = matched_df['ab_group'].value_counts().sort_index()
    control_users = ab_groups.get('6', 0)
    push_groups_count = matched_df[matched_df['ab_group'] != '6']['ab_group'].nunique()
    
    print(f"   ✅ 04_ab_analysis.ipynb - готово до A/B тестування з контрольною групою")
    print(f"      • Контрольна група: {control_users:,} користувачів")
    print(f"      • Push-групи: {push_groups_count} груп")
    
    if control_users > 100000:
        print(f"      • ✅ Контрольна група достатнього розміру для статистичної значущості")
    else:
        print(f"      • ⚠️ Контрольна група може бути малою для надійних висновків")
else:
    print(f"   ⚠️ 04_ab_analysis.ipynb - немає A/B груп або контрольної групи")

# Перевіряємо готовність для гео аналізу
if 'tier_final' in matched_df.columns and matched_df['tier_final'].nunique() > 1:
    tier_counts = matched_df['tier_final'].value_counts()
    tier_dict = {k: v for k, v in tier_counts.items() if k in ['Tier 1', 'Tier 2', 'Tier 3']}
    
    print(f"   ✅ 05_geo_tier_analysis.ipynb - готово з контрольною групою")
    print(f"      • Основні tier: {tier_dict}")
    
    # Перевіряємо чи є контрольна група в кожному tier
    if 'ab_group' in matched_df.columns:
        tier_control_counts = matched_df[matched_df['ab_group'] == '6']['tier_final'].value_counts()
        print(f"      • Контроль по tier: {tier_control_counts.to_dict()}")
else:
    print(f"   ⚠️ 05_geo_tier_analysis.ipynb - немає tier даних")

# Завжди готово для фінального звіту
print(f"   ✅ 06_final_report.ipynb - готово для фінального звіту з контрольною групою")

print("\n💡 РЕКОМЕНДАЦІЇ ДЛЯ A/B АНАЛІЗУ З КОНТРОЛЬНОЮ ГРУПОЮ:")

if 'intersection' in locals() and len(intersection) > 10000:
    print("   🎉 Відмінна якість матчингу - надійні результати A/B тестування")
elif 'intersection' in locals() and len(intersection) > 1000:
    print("   👍 Хороша якість матчингу - результати будуть статистично значущими")
else:
    print("   ⚠️ Обмежена якість матчингу - обережно з висновками")

# Аналіз конверсій
if 'has_deposit' in matched_df.columns and 'ab_group' in matched_df.columns:
    overall_conv_rate = matched_df['has_deposit'].mean() * 100
    
    control_df = matched_df[matched_df['ab_group'] == '6']
    push_df = matched_df[matched_df['ab_group'] != '6']
    
    if len(control_df) > 0 and len(push_df) > 0:
        control_conv = control_df['has_deposit'].mean() * 100
        push_conv = push_df['has_deposit'].mean() * 100
        lift = push_conv - control_conv
        
        print(f"   📊 Конверсії готові до аналізу:")
        print(f"      • Загальна конверсія: {overall_conv_rate:.2f}%")
        print(f"      • Контрольна група: {control_conv:.2f}%")
        print(f"      • Push-групи: {push_conv:.2f}%")
        print(f"      • Попередній lift: {lift:+.2f} п.п.")
        
        if abs(lift) > 0.1:
            if lift > 0:
                print(f"      • 🎯 Позитивний ефект push-сповіщень - детальний аналіз обов'язковий!")
            else:
                print(f"      • ⚠️ Негативний ефект push-сповіщень - потрібно розслідувати")
        else:
            print(f"      • 🤔 Мінімальний ефект - потрібен статистичний тест")

# Перевірка балансу груп
if 'ab_group' in matched_df.columns:
    ab_balance = matched_df['ab_group'].value_counts()
    push_groups = ab_balance[ab_balance.index != '6']
    
    if len(push_groups) > 0:
        balance_coefficient = push_groups.std() / push_groups.mean()
        control_to_push_ratio = ab_balance.get('6', 0) / push_groups.sum()
        
        print(f"   ⚖️ Баланс груп:")
        print(f"      • Коефіцієнт балансу push-груп: {balance_coefficient:.3f}")
        print(f"      • Співвідношення контроль/push: 1:{push_groups.sum()/ab_balance.get('6', 1):.1f}")
        
        if balance_coefficient < 0.1:
            print(f"      • ✅ Push-групи добре збалансовані")
        else:
            print(f"      • ⚠️ Push-групи не збалансовані - врахувати при аналізі")
        
        if 0.1 <= control_to_push_ratio <= 0.3:
            print(f"      • ✅ Оптимальне співвідношення контроль/експеримент")
        else:
            print(f"      • ⚠️ Неоптимальне співвідношення - може вплинути на статистичну потужність")

print(f"\n🧪 МОЖЛИВОСТІ АНАЛІЗУ З КОНТРОЛЬНОЮ ГРУПОЮ:")
print("   1. ✅ Розрахунок справжнього lift ефекту push-сповіщень")
print("   2. ✅ Статистичні тести значущості (t-test, chi-square)")
print("   3. ✅ Порівняння конверсій: контроль vs кожна push-група")
print("   4. ✅ Аналіз впливу інтенсивності push-ів на конверсії")
print("   5. ✅ Сегментований аналіз по tier та країнах")
print("   6. ✅ ROI розрахунки: вартість push-кампанії vs додатковий дохід")

print(f"\n🚀 СИСТЕМА ПОВНІСТЮ ГОТОВА ДО A/B ТЕСТУВАННЯ З КОНТРОЛЬНОЮ ГРУПОЮ!")
print("📈 Наступний крок: виконати 04_ab_analysis.ipynb для детального A/B аналізу")